In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [ ]:
def plot_prediction(model, dataset, idx):
    text = get_row(dataset, idx)['text']
    truth = np.array(get_row(dataset, idx)['labels'], dtype=bool)

    sks_code = converter.idx2sks(truth).split(';')
    sks_description = converter.idx2description(truth).split(';')
    description_list = converter.idx2description(np.ones_like(truth)).split(';')
    description_list_truth = converter.idx2description(truth).split(';')
    #description_list = [converter.sks2description_dict[sks] for sks in converter.popular_sks_codes]

    inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)

    inputs2 = tokenizer(text, return_tensors="pt")
    l = inputs2['input_ids'].shape[1]
    if l > 512:
        ratio = str((l - 512) / 512 * 100)
        ratio = ratio.split('.')[0]
        print(f"{'!'*54}\n[WARNING] {ratio}% of the text has been truncated [WARNING]\n{'!'*54}\n")
        
    print(text)
    print('='*100)
    for code, description in zip(sks_code, sks_description):
        print(f"{code} - {description}")
    print('='*100)

    input_cuda = {}
    for key in inputs.keys():
        input_cuda[key] = inputs[key].to('cuda')

    outputs = model(**input_cuda)
    logits = outputs[0].cpu().detach().numpy().flatten()
    x_ax = np.arange(len(truth))
    
    fig = go.Figure()

    fig.add_trace(go.Bar(x=x_ax, y=logits, 
                         marker_color='black', 
                         width=0.5, 
                         name="Predicted SKS code",
                         hovertext=description_list))

    fig.add_trace(go.Bar(x=x_ax[truth], y=logits[truth], 
                         marker_color='red', 
                         width=0.5, 
                         name="True SKS code",
                         hovertext=description_list_truth))

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=x_ax,
            ticktext=converter.popular_sks_codes,
            tickangle=90
        ),
        barmode='overlay'
    )

    fig.show()

"""    
def plot_error_pr_diagnosis ():

    error_of_diagnoses = np.array(errors).sum(axis=0) / len(errors) * 100
    x_ax = np.arange(len(error_of_diagnoses))

    fig = go.Figure(data=[
        go.Bar(x=x_ax, y=error_of_diagnoses)
    ])

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=x_ax,
            ticktext=converter.popular_sks_codes,
            tickangle=90
        ),
        yaxis_title="Errors (%)"
    )

    fig.show()

"""
def plot_training(trainer):
    history = trainer.state.log_history
    df_step = pd.DataFrame([step for step in history if 'loss' not in step])
    df_substep = pd.DataFrame([step for step in history if 'loss' in step])
    idxs = df_substep.epoch >= 1
    xlim = [0, max(df_step.epoch)]

    fig, (ax0, ax1) = plt.subplots(2, 1, sharex=True)
    fig.subplots_adjust(wspace=0, hspace=0)
    ax0.plot(df_substep.epoch, df_substep.loss, '.k')
    ax0.set_ylabel("Training loss")
    ax0.set_xlim(xlim)
    ax1.plot(df_substep.epoch[idxs], df_substep.loss[idxs] / df_substep[idxs].loss.iloc[0], '.r', label="Training loss")
    ax1.plot(df_step.epoch, df_step.eval_loss / df_step.eval_loss.iloc[0], 'o:b', label="Evaluation loss")
    ax1.legend()
    ax1.set_ylabel("Relative loss")

    fig, axs = plt.subplots(2, 1, sharex=True)
    fig.subplots_adjust(wspace=0, hspace=0)
    axs[0].plot(df_step.epoch, df_step.eval_f1, 'o:k')
    axs[0].set_ylabel("F1 metric")
    axs[0].set_xlabel("Epoch")
    axs[0].set_xlim(xlim)
    axs[1].plot(df_step.epoch, df_step.eval_loss, 'o:k')
    axs[1].set_ylabel("eval loss")
    axs[1].set_xlabel("Epoch")

def plot_sum_of_diagnoses ():
    sum_of_diagnoses = np.array(data['labels']).sum(axis=0)
    x_ax = np.arange(len(sum_of_diagnoses))

    fig = go.Figure(data=[
        go.Bar(x=x_ax, y=sum_of_diagnoses)
    ])

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=x_ax,
            ticktext=converter.popular_sks_codes,
            tickangle=90
        )
    )

    fig.show()
    
def plot_errors():
    
    def find_error(examples):
        text = examples['text']
        inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
        input_cuda = {}
        for key in inputs.keys():
            input_cuda[key] = inputs[key].to('cuda')
        outputs = model(**input_cuda)

        truth = np.array(examples['labels'], dtype=bool)
        prediction = outputs['logits'] > 0

        pred = prediction.cpu().detach().numpy()[0]


        out = {'truth' : truth,
               'prediction' : pred,
               'error' : truth != pred
              }
        return out

    error_dataset = dataset['test'].map(find_error, remove_columns=['sks_koder', 'journal_note', 'text', 'sks_beskrivelser', 'journal_note', 'labels'])

    pd = error_dataset.to_pandas()

    pd_errors = pd[[error.any() for error in pd.error]]

    pd_errors = pd_errors.assign(pred_should_be_false = pd_errors.prediction * pd_errors.error)
    pd_errors = pd_errors.assign(pred_should_be_true = ~pd_errors.prediction * pd_errors.error)

    sum_errors = np.array(list(pd_errors.error)).sum(axis=0)
    sum_pred_should_be_false = np.array(list(pd_errors.pred_should_be_false)).sum(axis=0)
    sum_pred_should_be_true = np.array(list(pd_errors.pred_should_be_true)).sum(axis=0)
    sum_pr_sks = np.array(list(pd.truth)).sum(axis=0)

    a1 = 100 * sum_pred_should_be_true / sum_pr_sks
    a2 = 100 * sum_pred_should_be_false / sum_pr_sks

    b1 = 100 * sum_pred_should_be_true / len(pd)
    b2 = 100 * sum_pred_should_be_false / len(pd)

    # Create subplots: 3 rows, 1 column
    fig = make_subplots(rows=3, cols=1, shared_xaxes=True)

    # Add traces
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=sum_pred_should_be_true, name='Prediction should be true', marker_color='blue'), row=1, col=1)
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=sum_pred_should_be_false, name='Prediction should be false', marker_color='green'), row=1, col=1)
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=sum_pred_should_be_false + sum_pred_should_be_true, name='Total errors', marker_color='black'), row=1, col=1)

    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=a1, name='Prediction should be true', marker_color='blue'), row=2, col=1)
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=a2, name='Prediction should be false', marker_color='green'), row=2, col=1)
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=a1 + a2, name='All errors', marker_color='black'), row=2, col=1)

    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=b1, name='Prediction should be true', marker_color='blue'), row=3, col=1)
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=b2, name='Prediction should be false', marker_color='green'), row=3, col=1)
    fig.add_trace(go.Bar(x=converter.popular_sks_codes, y=b1 + b2, name='All errors', marker_color='black'), row=3, col=1)

    # Update yaxis properties
    fig.update_yaxes(title_text="Nbr of errors", row=1, col=1)
    fig.update_yaxes(title_text="% of errors pr sks code", row=2, col=1)
    fig.update_yaxes(title_text="% of errors", row=3, col=1)

    # Update layout properties
    fig.update_layout(height=1200, width=1000)#, title_text="Stacked Subplots with Shared X-Axes")
    
    fig.show()


In [ ]:
"""def get_errors(max_iters=1000, verbose=True):

    def get_error(idx):
        l = len(dataset["test"]['text'])
        text = dataset["test"]['text'][idx]
        truth = np.array(dataset["test"]['labels'][idx], dtype=bool)

        too_long = False
        inputs = tokenizer(text, return_tensors="pt", padding="max_length", truncation=True)
        inputs2 = tokenizer(text, return_tensors="pt")
        if inputs2['input_ids'].shape[1] > 512:
            too_long = True

        input_cuda = {}
        for key in inputs.keys():
            input_cuda[key] = inputs[key].to('cuda')

        outputs = model(**input_cuda)

        logits = outputs[0].cpu().detach().numpy().flatten()

        guess = logits > 0

        return truth != guess, too_long

    max_iters = min(max_iters, len(dataset['test']))
    errors = []
    too_longs = []
    for idx in range(max_iters):
        error, too_long = get_error(idx)
        errors.append(error)
        too_longs.append(too_long)

        if verbose and idx % int(max_iters / 10):
            print(f"{100*idx / max_iters:.1f}%")

    return errors, too_longs"""

In [10]:
from datasets.iterable_dataset import IterableDataset
from datasets.arrow_dataset import Dataset

def get_row(dataset, idx):
    if type(dataset) is IterableDataset:
        for i, row in enumerate(tokenized_datasets['test']):
            if i == idx:
                return row
        raise IndexError(f"Index {idx} is out of bounds")

    elif type(dataset) is Dataset:
        return dataset[idx]
    
    else:
        raise Exception(f"Dataset of type {type(dataset)} not supported")
        
        
    tokenized_datasets["test"]['text'][idx]